# Домашнее задание № 3. Исправление опечаток

In [1]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
from nltk import sent_tokenize
punctuation += "«»—…“”"
punct = set(punctuation)
from sklearn.metrics import classification_report, accuracy_score
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from collections import Counter

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [2]:
bad = open('data/sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('data/correct_sents.txt', encoding='utf8').read().splitlines()

In [3]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [token.strip(punctuation) for token in tokens_1]
    tokens_2 = [token.strip(punctuation) for token in tokens_2]
    
    tokens_1 = [token for token in tokens_1 if token]
    tokens_2 = [token for token in tokens_2 if token]
    
    assert len(tokens_1) == len(tokens_2)
    
    return list(zip(tokens_1, tokens_2))

## 1. Доп. ранжирование по вероятности (3 балла)

Дополните get_closest_hybrid_match в семинаре так, чтобы из кандадатов с одинаковым расстоянием редактирования выбиралось наиболее вероятное.

In [4]:
import textdistance
from difflib import get_close_matches
from tqdm import tqdm

In [5]:
corpus = open('data/wiki_data.txt', encoding='utf8').read()

In [6]:
vocab = Counter(re.findall('\w+', corpus.lower()))

word2id = list(vocab.keys())
id2word = {i:word for i, word in enumerate(vocab)}


vec = CountVectorizer(analyzer='char', max_features=10000, ngram_range=(1,3))
X = vec.fit_transform(vocab)

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Alex\AppData\Local\Temp\ipykernel_29444\2167344717.py:1: SyntaxWarning: invalid escape sequence '\w'
  vocab = Counter(re.findall('\w+', corpus.lower()))


In [13]:
def get_closest_match_with_metric(text, lookup,topn=20, metric=textdistance.levenshtein):
    # Counter можно использовать и с не целыми числами
    similarities = Counter()
    
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
        
    return similarities.most_common(topn)

def get_closest_hybrid_match(text, X, vec, topn=3, metric=textdistance.damerau_levenshtein):
    candidates = get_closest_match_vec(text, X, vec, topn*4)
    lookup = [cand[0] for cand in candidates]
    closest = get_closest_match_with_metric(text, lookup, topn, metric=metric)

    for i in range(1, len(closest)-1):
        word = closest[i][0]
        s_word = closest[i][1]
        prev_word = closest[i-1][0]
        s_prev_word = closest[i-1][1]
        if s_word == s_prev_word:
            p_word = P(word)
            p_prev = P(prev_word)
            if p_word>p_prev:
                closest.pop(i-1)
            else:
                closest.pop(i)
                
    return (closest[0])

N = sum(vocab.values())

def P(word, N=N):
    return vocab[word] / N

def predict_mistaken(word, vocab):
    return 0 if word in vocab else 1

def get_closest_match_vec(text, X, vec, topn=20):
    v = vec.transform([text])
    
    similarities = cosine_distances(v, X)[0]
    topn = similarities.argsort()[:topn] 
    
    return [(id2word[top], similarities[top]) for top in topn]


In [9]:
get_closest_hybrid_match('сонце', X, vec)

('солнце', 0.8333333333333334)

In [10]:
get_closest_hybrid_match('лесница', X, vec)

('лестница', 0.875)

In [11]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1],get_closest_hybrid_match(pair[1], X, vec)[0])
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████| 915/915 [08:55<00:00,  1.71it/s]


In [12]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8552276138069035
0.48524844720496896
0.09004249454461927


## 2.  Symspell (7 баллов)

Реализуйте алгоритм Symspell. Он похож на алгоритм Норвига, но проще и быстрее. Он основан только на одной операции - удалении символа. Описание алгоритма по шагам:

1) Составляется словарь правильных слов  
2) На основе словаря правильных слов составляется словарь удалений - для каждого правильного слова создаются все варианты удалений и создается словарь, где ключ - слово с удалением, а значение - правильное слово  (!) 
3) Для выбора исправления для слова с опечаткой генерируются все варианты удаления, из них выбираются те, что есть в словаре удалений, построенного на шаге 2. Слово с опечаткой заменяется на правильное слово, соответствующее варианту удаления  
4) Если в словаре удалений есть несколько вариантов, то выбирается удаление, которому соответствует наиболее вероятное правильное слово  


Оцените качество полученного алгоритма теми же тремя метриками.

In [7]:
def del_char(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    return deletes

In [8]:
#cоставляется словарь правильных слов

good_words = set(w for w in vocab)

#cоставляется словарь удалений

del_dict = dict()
for word in good_words:
    del_words = del_char(word)
    for dword in del_words:
        if dword not in del_dict:
            del_dict[dword] = {word}
        else:
            del_dict[dword].add(word)

In [9]:
def del_match(del_dict, word):
    candids = []
    for dels in del_char(word):
        if dels in del_dict:
            candids.extend(del_dict[dels])
    return candids

In [10]:
def check_Symspell(word):
    if word in good_words:
        return word
    else:
        candids = del_match(del_dict, word)
        probs_c = dict()
        if candids!=[]:
            for c in candids:
                probs_c [c] = P(c)
                probs_dict = {k: v for k, v in sorted(probs_c.items(), key=lambda item: item[1], reverse=True)}
            res = list(probs_dict.keys())[0]
        elif candids == []:
            res = word
        print(word, res)
        return res

In [14]:
check_Symspell('сабака')

сабака собака


'собака'

In [15]:
check_Symspell('собака')

'собака'

In [16]:
check_Symspell('гаворить')

гаворить говорить


'говорить'

In [17]:
check_Symspell('жызнь')

жызнь жизнь


'жизнь'

In [18]:
mistakes = []
total_mistaken = 0
mistaken_fixed = 0

total_correct = 0
correct_broken = 0

total = 0
correct = 0

cashed = {}
for i in tqdm(range(len(true))):
    word_pairs = align_words(true[i], bad[i])
    for pair in word_pairs:
        if predict_mistaken(pair[1], vocab):
            pred = cashed.get(pair[1],check_Symspell(pair[1]))
            cashed[pair[1]] = pred
        else:
            pred = pair[1]
        
            
        if pred == pair[0]:
            correct += 1
        else:
            mistakes.append((pair[0], pair[1], pred))
        total += 1
            
        if pair[0] == pair[1]:
            total_correct += 1
            if pair[0] != pred:
                correct_broken += 1
        else:
            total_mistaken += 1
            if pair[0] == pred:
                mistaken_fixed += 1


100%|██████████| 915/915 [00:00<00:00, 20914.38it/s]

симпатичнейшое симпатичнейшое
шпионское шпионской
гламурный гламурным
superheadz superheadz
clap camp
опофеозом апофеозом
услышанная услышанная
пояним эпоним
полчатся полчатся
язычки язычок
оччччень оччччень
нащщот нащщот
сетуют сетует
отсуствие отсуствие
основая основан
аварийно-спасательных аварийно-спасательных
дорожно-транспортных дорожно-транспортных
нарасно набрано
вобщем вообще
ящека щенка
почте.ру почте.ру
предлагю предлагю
сегодяшнее сегодяшнее
хороше хорошо
выходгых выходных
рите риме
снятся снялся
патаму панаму
пытаюсь пытаясь
лчше чаше
дубраве дубрава
люминала люминала
поффтыкав поффтыкав
билетным билетные
компютерная компютерная
чтото тотом
выговаривает выговаривает
мошный мощный
кредиток кредитов
хороше хорошо
капулетти капулетти
кормилицу кормилица
учавствовать учавствовать
седня седан
притащиться притащиться
оплеух оплеух
отвественность отвественность
пивоварах пивоварах
канешна казнена
4:0 400
начальнег начальное
зажог залог
павзрослому павзрослому
предудущую предыдущу

In [19]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.8560280140070035
0.19953416149068323
0.04685884920179166
